In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import datetime

from Data import DataCollector
from Metrics import Metrics

In [80]:

tik = ["AAPL", "MSFT", "NVDA", "AMZN", "GOOG", "TSLA"]
beg = "2020-01-01"
end = "2024-12-31"

data = DataCollector(tik, beg, end)
rt = data.get_returns()
rt

[*********************100%***********************]  6 of 6 completed


Ticker,AAPL,AMZN,GOOG,MSFT,NVDA,TSLA
Date,,,,,,
2020-01-03,-0.009722,-0.012139,-0.004907,-0.012452,-0.016006,0.029633
2020-01-06,0.007968,0.014886,0.024657,0.002585,0.004194,0.019255
2020-01-07,-0.004703,0.002092,-0.000624,-0.009118,0.012107,0.038801
2020-01-08,0.016086,-0.007809,0.007880,0.015929,0.001876,0.049205
2020-01-09,0.021241,0.004799,0.011044,0.012493,0.010983,-0.021945
...,...,...,...,...,...,...
2024-12-23,0.003065,0.000622,0.015703,-0.003092,0.036897,0.022657
2024-12-24,0.011478,0.017729,0.008062,0.009374,0.003938,0.073572
2024-12-26,0.003176,-0.008732,-0.002379,-0.002777,-0.002068,-0.017630


In [94]:
months = rt.resample("MS").last().index

portfolio = []

for i in range(12, len(months) - 1):
    
    beg = months[i - 12]
    end = months[i - 1]
    # print(beg, end, months[i])
    rt_i = rt[(rt.index >= beg) & (rt.index < end)]
    mom = (1 + rt_i).prod() - 1
    w = mom.rank() - mom.rank().sum() / len(mom)
    w_long = w[w > 0].sum()
    ct = 1 / w_long
    w = ct * w
    # print(months[i], months[i +1])
    ri = rt[(rt.index >= months[i]) & (rt.index < months[i + 1])]
    ri_cum = (1+ ri).cumprod()
    pi = ri * w
    
    pt = pi.sum(axis = 1)
    
    portfolio.append(pt)


portfolio = pd.concat(portfolio)

In [120]:
rt_portfolio = (1 + portfolio).cumprod() - 1
rt_portfolio_cum = (1 + portfolio).prod() - 1
import textwrap
metrics = Metrics(portfolio)
mu = metrics.mean()
st = metrics.std()
sr = metrics.sharpe()

print(metrics.report())

print("Cumulative returns: ", rt_portfolio_cum)
plt.plot(rt_portfolio)
portfolio

NameError: name 'textwrap' is not defined

In [ ]:
# simple example for rebalance weights
returns = pd.DataFrame({
    "AAPL": [0.01, -0.02, 0.03, 0.01, 0.02],
    "MSFT": [0.02,  0.01, -0.01, 0.00, 0.01],
    "GOOG": [-0.01, 0.02, 0.01, 0.01, -0.02]
}, index=pd.date_range("2023-01-01", periods=5))

# --- Step 2: Initial equal weights ---
w0 = np.array([1/3, 1/3, 1/3])

# --- Step 3: Compute cumulative growth factors ---
cum_growth = (1 + returns).cumprod()W

# This represents the value of $1 invested in each asset since t=0
print(cum_growth)

                AAPL      MSFT      GOOG
2023-01-01  1.010000  1.020000  0.990000
2023-01-02  0.989800  1.030200  1.009800
2023-01-03  1.019494  1.019898  1.019898
2023-01-04  1.029689  1.019898  1.030097
2023-01-05  1.050283  1.030097  1.009495


In [88]:
import pandas as pd
import numpy as np

signals = pd.DataFrame({
    '2023-01-01': [0.1, 0.2, 0.05, -0.1, -0.3],
    '2023-01-02': [0.3, 0.1, -0.05, -0.2, 0.0]
}, index=['A','B','C','D','E'])

def compute_weights(sig):
    ranks = sig.rank()
    demeaned = ranks - ranks.mean()
    print(demeaned)
    pos_sum = demeaned[demeaned > 0].sum()
    c_t = 1 / pos_sum        # ensures $1 long and $1 short
    print(ct)
    return c_t * demeaned

weights = signals.apply(compute_weights, axis=0)

# Sanity checks
for t in weights.columns:
    w = weights[t]
    print(f"{t}: long={w[w>0].sum():.2f}, short={w[w<0].sum():.2f}, net={w.sum():.2f}")


A    1.0
B    2.0
C    0.0
D   -1.0
E   -2.0
Name: 2023-01-01, dtype: float64
0.5
A    2.0
B    1.0
C   -1.0
D   -2.0
E    0.0
Name: 2023-01-02, dtype: float64
0.5
2023-01-01: long=1.00, short=-1.00, net=0.00
2023-01-02: long=1.00, short=-1.00, net=0.00
